In [ ]:
# загрузка библиотеки для img2table + PaddleOCR
%pip install img2table[paddle]
# для совместимости требуется protobuf не выше 3.20.2
%pip install --force-reinstall -v 'protobuf==3.20.2'

In [1]:
# импорт библиотек
import pandas as pd
import cv2

from img2table.document import Image as Image2
from img2table.ocr import PaddleOCR

from PIL import Image

from IPython.display import display

In [2]:
# функция для распознавания таблицы на изображении

def itt(ocr, image, min_confidence=50, rotation=False):
  # загрузка изображения с текстом
  doc2 = Image2(image, detect_rotation=rotation)

  # извлечение таблицы из изображения
  extracted_table = doc2.extract_tables(ocr=ocr,
                                        implicit_rows=True,
                                        borderless_tables=False,
                                        min_confidence=min_confidence)

  try:
    # перевод результата в датасет
    data = extracted_table[0].df
    return data
  except IndexError:
    print('Таблица не распознана')
    return pd.DataFrame()

In [3]:
# функция предобработки изображения 0
# оставлен лучший вариант с переводом в оттенки серого и двойной фильтрацией
# остальное закомментировано
def prep0(img_path, output_path):
    image = cv2.imread(img_path)
    # размытие по Гаусу
    #image = cv2.GaussianBlur(image, (5, 5), 0)
    # срединное размытие
    #image = cv2.medianBlur(image, 3)
    # двусторонняя фильтрация
    image = cv2.bilateralFilter(image,9,75,75)
    # адаптивный порог (через оттенки серого)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    cv2.imwrite(output_path, image)
    return

In [4]:
# функция предобработки изображения 1
def prep1(img_path, output_path):
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    cv2.imwrite(output_path, sobel)

In [5]:
# функция предобработки изображения 2
def prep2(img_path, output_path):
    image = cv2.imread(img_path)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    limg = cv2.merge((cl, a, b))
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2.imwrite(output_path, enhanced_img)

In [6]:
# функция сравнения результатов распознавания с реальными значениями
# df1 - датасет с реальными значениями
# df2 - датасет с результатами распознавания
def compare_tables(df1, df2):
    # считаем, что правильной является 1 таблица
    
    num_matching_cells = (df1 == df2).sum().sum()

    total_cells = df1.size
    
    accuracy = num_matching_cells/total_cells
    return num_matching_cells, total_cells, accuracy

In [7]:
# применение к изображению tablica1.jpg функции предобработки 0,
# затем функции предобработки 2 (как самой лучшей комбинации функций предобработки) 
# сохранение результата в tabl_temp.jpg
prep0('tablica1.jpg', 'tabl_temp.jpg')
prep2('tabl_temp.jpg', 'tabl_temp.jpg')

# создание OCR из PaddleOCR
ocr = PaddleOCR(lang='ru')

# вызов функции распознавания таблиц (возвращает датафрейм df)
df = itt(ocr, 'tabl_temp.jpg', 50, False)

# переименование столбцов в df, как в df_test
df.columns = [str(x) for x in list(range(9))]

# отображение распознанной информации
print('Распознанная информация:')
display(df)

# загрузка и отображение проверочной информации
df_test = pd.read_csv('tablica1.csv')
print('Проверочная информация:')
display(df_test)

# удаление строк с заголовками
df_test = df_test.drop(labels=0, axis = 0)
df = df.drop(labels=0, axis=0)

# вызов функции сравнения результата распознавания
num_matching_cells, total_cells, accuracy = compare_tables(df_test, df)

# вывод результата сверки
print(f'Корректно распознанных ячеек: {accuracy*100:0.2f} %.')

Распознанная информация:


,0,1,2,3,4,5,6,7,8
0,ата,Вид\nдOH-EE,Кoлвo,Дата,Вид\nдOHBа,олво,Дата,ид\nDH-EE,Кoлbo
1,14022006,крд\nбв,42о,15.O7,крд\nбв,дђо,04.08.\n2о,крд\nбв,ђо
2,None,кргд\nбв,зђо,1l0.206,крд\nбв,Ао,26.12.2018,крд\nбв,4зо
3,30.10.2014,крд\nбв,ђо,22:2о16,крд\nбв,дђо,"29,03.2019",бв\nкрд,Ађо
4,13.08.\n2о5,крд\nбв,до,2106.2017,крд\nбв,Ађо,11102022,кргд\nбв,дђо
5,3o.10.\n2о,крд\nбв,Ађо,None,None,None,None,None,None


Проверочная информация:


,0,1,2,3,4,5,6,7,8
0,Дата,Вид дон-ва,Количество,Дата,Вид дон-ва,Количество,Дата,Вид дон-ва,Количество
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,NaN,NaN,NaN,NaN,NaN,NaN


Корректно распознанных ячеек: 4.44 %.
